In [1]:
# Importing libraries
from pyedlut import simulation_wrapper as pyedlut
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy.stats import norm
import seaborn as sns
import scipy.stats as stats 
from scipy.stats import f_oneway
from scipy.ndimage import gaussian_filter
import matplotlib.ticker as ticker

## Importing functions
import sys
sys.path.insert(1, '../data')
import figure_4_functions as fun
import simulation_function as sim
import plot_spikes_function as scatter

In [2]:
## Creating plots directory
RESULTS_DIR = "../results/figures_plots"
FIGURE_4_RESULTS_DIR = os.path.join(RESULTS_DIR, 'figure_4')
os.makedirs(FIGURE_4_RESULTS_DIR, exist_ok = True)


# OVERVIEW

In [3]:
## Creating default parameters dictionary
my_dict_default = {"fraction":0, 
           "pattern": 0,
           "sigma": 5, 
            "seed": 0,
            "poisson_seed": 0,
            "seed_noise": 0,
            "noise": 0.1, 
            "mf_goc_w": 0.0,
            "grc_goc_w": 0.0,
            "num_patterns": 8,
            "duration_pattern": 0.2,
            "interval": 0.2,
            "firing_rate": 50.0, 
            "step": 3
}

In [4]:
## Creating dictionary to initially visualize the spikes
dict_fig_overview = {
    **my_dict_default,
    "fraction": 3,
    "num_patterns": 20, 
    "mf_goc_w": 0.00,
    "grc_goc_w": 0.00,
    "duration_pattern": 0.08, 
    "interval": 0.07
}

In [ ]:
## Retrieving activity
mf_activity = fun.mf_activity_overview(dict_fig_overview)
## Running simulation
oi, ot, _, _ = sim.simulation(dict_fig_overview, mf_activity)
## Plotting spikes
fig = scatter.plot_spikes(oi, ot, dict_fig_overview, 0.0, 10*dict_fig_overview["duration_pattern"])

In [ ]:
dict_fig_overview = {
    **my_dict_default,
    "fraction": 4,
    "sigma": 40, 
    "num_patterns": 20, 
    "mf_goc_w": 0.00,
    "grc_goc_w": 0.00,
    "duration_pattern": 0.08, 
    "interval": 0.07
}

## Retrieving activity
mf_activity = fun.mf_activity_overview(dict_fig_overview)
## Running simulation
oi, ot, _, _ = sim.simulation(dict_fig_overview, mf_activity)
## Plotting spikes
fig = scatter.plot_spikes(oi, ot, dict_fig_overview, 0.0, 10*dict_fig_overview["duration_pattern"])

## Selection and combination of two different patterns

In [7]:
## Creating dictionaries for each stimulus with and withou inhibition
dict_1_without = {
    **my_dict_default,
    "fraction": 2,
    "pattern": 2, 
}

dict_2_without = {
    **my_dict_default,
    "fraction": 4,
    "pattern": 5, 
}

dict_1_with = {
    **my_dict_default,
    "fraction": 2,
    "pattern": 2, 
    "mf_goc_w": 0.1,
    "grc_goc_w": 0.015
}

dict_2_with = {
    **my_dict_default,
    "fraction": 4,
    "pattern": 5, 
    "mf_goc_w": 0.1,
    "grc_goc_w": 0.015
}

In [ ]:
## Running simulations for each stimulus

mf_activity = fun.mf_activity(dict_1_without)
_, _, matrix_1, _ = sim.simulation(dict_1_without, mf_activity)

mf_activity = fun.mf_activity(dict_1_with)
_, _, matrix_1_inh, _  = sim.simulation(dict_1_with, mf_activity)

mf_activity = fun.mf_activity(dict_2_without)
_, _, matrix_2, _  = sim.simulation(dict_2_without, mf_activity)

mf_activity = fun.mf_activity(dict_2_with)
_, _, matrix_2_inh, _  = sim.simulation(dict_2_with, mf_activity)

## Running simulations for the combination of both stimuli
mf_activity = fun.mf_activity_combined(dict_1_without, dict_2_without)
_, _, matrix_3, _  = sim.simulation(dict_1_without, mf_activity)

mf_activity = fun.mf_activity_combined(dict_1_with, dict_2_with)
_, _, matrix_3_inh, _  = sim.simulation(dict_1_with, mf_activity)

In [9]:
## Calculating facilitated, emergent, suppressed and unchanged cells
## Without inhibition case

m1 = np.sum(matrix_1[:,8:], axis = 1)/matrix_1[:,8:].shape[1]
m2 = np.sum(matrix_2[:,8:], axis = 1)/matrix_2[:,8:].shape[1]
m3 = np.sum(matrix_3[:,8:], axis = 1)/matrix_3[:,8:].shape[1]

act1 = np.where(m1 != 0)[0]
act2 = np.where(m2 != 0)[0]
th1 = matrix_1[act1,:].mean()
th2 = matrix_2[act2,:].mean()
std1 = matrix_1[act1,:].std()
std2 = matrix_2[act2,:].std()

if th1 > th2:
    th = th2 
else:
    th = th1 

m1_th = m1>th
m2_th = m2>th
m3_th = m3>th

facilitated = []
unchanged = []
supressed = []
emergent = []

for i in range(matrix_1.shape[0]):
    
    if m1[i] > m2[i]:
        maximo = m1[i]
        minimo = m2[i]
    else:
        maximo = m2[i]
        minimo = m1[i]
    
    if (m1_th[i] == False)&(m2_th[i] == False):
        if (m3_th[i] == True):
            emergent.append(i)
        else: 
            unchanged.append(i)
        
    else:
        if (m3_th[i] == False):
            supressed.append(i)
        else:
            if m3[i] > maximo: 
                facilitated.append(i)
            else:
                unchanged.append(i)
            


In [ ]:
## Calculating facilitated, emergent, suppressed and unchanged cells
## With inhibition case

m1 = np.sum(matrix_1_inh[:,8:], axis = 1)/matrix_1_inh[:,8:].shape[1]
m2 = np.sum(matrix_2_inh[:,8:], axis = 1)/matrix_2_inh[:,8:].shape[1]
m3 = np.sum(matrix_3_inh[:,8:], axis = 1)/matrix_3_inh[:,8:].shape[1]

act1 = np.where(m1 != 0)[0]
act2 = np.where(m2 != 0)[0]
th1 = matrix_1_inh[act1,:].mean()
th2 = matrix_2_inh[act2,:].mean()
std1 = matrix_1_inh[act1,:].std()
std2 = matrix_2_inh[act2,:].std()

if th1 > th2:
    th = th2 
else:
    th = th1 


m1_th = m1>th
m2_th = m2>th
m3_th = m3>th

facilitated_inh = []
unchanged_inh = []
supressed_inh = []
emergent_inh = []
num = 0
for i in range(matrix_1_inh.shape[0]):
    
    if m1[i] > m2[i]:
        maximo = m1[i]
        minimo = m2[i]
    else:
        maximo = m2[i]
        minimo = m1[i]
    
    
    if (m1_th[i] == False)&(m2_th[i] == False):
        if (m3_th[i] == True):
            emergent_inh.append(i)

        else: 
            unchanged_inh.append(i)

        
    else:
        if (m3_th[i] == False):
            supressed_inh.append(i)

            if m1[i] > m2[i]:
                num += 1
        else:
            if m3[i] > maximo: 

                facilitated_inh.append(i)
            else:

                unchanged_inh.append(i)
    print("")
            
            
     

In [11]:
## Number of cells per group

In [ ]:
[len(facilitated_inh), len(emergent_inh), len(unchanged_inh), len(supressed_inh)]

In [ ]:
[len(facilitated), len(emergent), len(unchanged), len(supressed)]

In [14]:
text = str(dict_1_with["fraction"]) + '_' + str(dict_2_with["fraction"]) + '_' + str(dict_1_with["pattern"]) + '_' + str(dict_2_with["pattern"]) + '_sigma_' + str(20)

### Pie chart

In [ ]:
## Plotting pie chart to visualize amount of cells inside each category
plt.rcParams['figure.figsize'] = (6,17)
plt.rcParams["savefig.facecolor"] = "white"
labels = ['Facilitated', 'Supressed', 'Unchanged','Emergent']
sizes1 = [len(facilitated_inh), len(supressed_inh), len(unchanged_inh), len(emergent_inh)]
sizes2 = [len(facilitated), len(supressed), len(unchanged), len(emergent)]
fig, (ax1, ax2) = plt.subplots(nrows = 2, ncols = 1)
ax1.set_title('Control', fontsize = 19)
ax2.set_title('DART', fontsize = 19, color = 'red')


colors = [20, 100, 150, 200] # grayscale data
ax1.pie(sizes1, colors = [str(x/255) for x in colors], wedgeprops = {"linewidth": 2, "edgecolor": "black"})
ax2.pie(sizes2, colors = [str(x/255) for x in colors], wedgeprops = {"linewidth": 2, "edgecolor": "black"})
ax1.legend(loc=(0.28, -0.28), fontsize = 17, labels=labels)


fig.savefig(FIGURE_4_RESULTS_DIR + '/pie_chart' + text + '.png', bbox_inches="tight", dpi = 200)


# TRIALS

In [16]:
## Setting dictionaries for each stimulus to run trials with different noise

dict_1_without = {
    **my_dict_default,
    "fraction": 2,
    "pattern": 2, 
    "num_patterns": 12
}

dict_2_without = {
    **my_dict_default,
    "fraction": 4,
    "pattern": 5, 
    "num_patterns": 12
}

dict_1_with = {
    **my_dict_default,
    "fraction": 2,
    "pattern": 2, 
    "mf_goc_w": 0.1,
    "grc_goc_w": 0.015,
    "num_patterns": 12
}

dict_2_with = {
    **my_dict_default,
    "fraction": 4,
    "pattern": 5, 
    "mf_goc_w": 0.1,
    "grc_goc_w": 0.015,
    "num_patterns": 12
}

In [ ]:
## Single and combined stimuli presentations without inhibition for 10 trials

mf_activity = fun.mf_activity_trials(dict_1_without)
_, _, matrix_trial_1, _  = sim.simulation(dict_1_without, mf_activity)
mf_activity = fun.mf_activity_trials(dict_2_without)
_, _, matrix_trial_2, _  = sim.simulation(dict_2_without, mf_activity)
mf_activity = fun.mf_activity_combined_trials(dict_1_without, dict_2_without)
_, _, matrix_trial_3, _  = sim.simulation(dict_1_without, mf_activity)

mean_before = matrix_trial_1[:,:8].mean()
matrix_trial_1 = matrix_trial_1/mean_before
mean_before = matrix_trial_2[:,:8].mean()
matrix_trial_2 = matrix_trial_2/mean_before
mean_before = matrix_trial_3[:,:8].mean()
matrix_trial_3 = matrix_trial_3/mean_before

times = 10


matrix1 = np.zeros((matrix_trial_1.shape[0], matrix_trial_1.shape[1], times))
matrix2 = np.zeros((matrix_trial_2.shape[0], matrix_trial_2.shape[1], times))
matrix3 = np.zeros((matrix_trial_3.shape[0], matrix_trial_3.shape[1], times))

matrix1[:,:,0] = matrix_trial_1
matrix2[:,:,0] = matrix_trial_2
matrix3[:,:,0] = matrix_trial_3


for i in range(times-1):

    dict_1_without = {
        **dict_1_without,
        "poisson_seed": i+1
    }

    dict_2_without = {
        **dict_2_without,
        "poisson_seed": i+1
    }

    mf_activity = fun.mf_activity_trials(dict_1_without)
    _, _, matrix_trial_1, _  = sim.simulation(dict_1_without, mf_activity)

    mf_activity = fun.mf_activity_trials(dict_2_without)
    _, _, matrix_trial_2, _  = sim.simulation(dict_2_without, mf_activity)

    mf_activity = fun.mf_activity_combined_trials(dict_1_without, dict_2_without)
    _, _, matrix_trial_3, _  = sim.simulation(dict_1_without, mf_activity)


    mean_before = matrix_trial_1[:,:8].mean()
    matrix_trial_1 = matrix_trial_1/mean_before
    mean_before = matrix_trial_2[:,:8].mean()
    matrix_trial_2 = matrix_trial_2/mean_before
    mean_before = matrix_trial_3[:,:8].mean()
    matrix_trial_3 = matrix_trial_3/mean_before

    matrix1[:,:,i+1] = matrix_trial_1
    matrix2[:,:,i+1] = matrix_trial_2
    matrix3[:,:,i+1] = matrix_trial_3

In [ ]:
## Single and combined stimuli presentations with inhibition for 10 trials

mf_activity = fun.mf_activity_trials(dict_1_with)
_, _, matrix_trial_1, _  = sim.simulation(dict_1_with, mf_activity)
mf_activity = fun.mf_activity_trials(dict_2_with)
_, _, matrix_trial_2, _  = sim.simulation(dict_2_with, mf_activity)
mf_activity = fun.mf_activity_combined_trials(dict_1_with, dict_2_with)
_, _, matrix_trial_3, _  = sim.simulation(dict_1_with, mf_activity)

mean_before = matrix_trial_1[:,:8].mean()
matrix_trial_1 = matrix_trial_1/mean_before
mean_before = matrix_trial_2[:,:8].mean()
matrix_trial_2 = matrix_trial_2/mean_before
mean_before = matrix_trial_3[:,:8].mean()
matrix_trial_3 = matrix_trial_3/mean_before

times = 10

matrix1_inh = np.zeros((matrix_trial_1.shape[0], matrix_trial_1.shape[1], times))
matrix2_inh = np.zeros((matrix_trial_2.shape[0], matrix_trial_2.shape[1], times))
matrix3_inh = np.zeros((matrix_trial_3.shape[0], matrix_trial_3.shape[1], times))

matrix1_inh[:,:,0] = matrix_trial_1
matrix2_inh[:,:,0] = matrix_trial_2
matrix3_inh[:,:,0] = matrix_trial_3


for i in range(times-1):
    
    dict_1_with = {
        **dict_1_with,
        "poisson_seed": i+1
    }

    dict_2_with = {
        **dict_2_with,
        "poisson_seed": i+1
    }

    mf_activity = fun.mf_activity_trials(dict_1_with)
    _, _, matrix_trial_1, _  = sim.simulation(dict_1_with, mf_activity)

    mf_activity = fun.mf_activity_trials(dict_2_with)
    _, _, matrix_trial_2, _  = sim.simulation(dict_2_with, mf_activity)

    mf_activity = fun.mf_activity_combined_trials(dict_1_with, dict_2_with)
    _, _, matrix_trial_3, _  = sim.simulation(dict_1_with, mf_activity)
       
    mean_before = matrix_trial_1[:,:8].mean()
    matrix_trial_1 = matrix_trial_1/mean_before
    mean_before = matrix_trial_2[:,:8].mean()
    matrix_trial_2 = matrix_trial_2/mean_before
    mean_before = matrix_trial_3[:,:8].mean()
    matrix_trial_3 = matrix_trial_3/mean_before
    

    matrix1_inh[:,:,i+1] = matrix_trial_1
    matrix2_inh[:,:,i+1] = matrix_trial_2
    matrix3_inh[:,:,i+1] = matrix_trial_3
    


## EMERGENT GRCS PLOTS

In [19]:
## Calculating emergent cells for the case without inhibition
all_emergent1 = np.mean(matrix1[emergent, :,:], axis = 0)
all_emergent2 = np.mean(matrix2[emergent, :,:], axis = 0)
all_emergent3 = np.mean(matrix3[emergent, :,:], axis = 0)

mean1 = np.mean(all_emergent1.T, axis=0)
mean2 = np.mean(all_emergent2.T, axis=0)
mean3 = np.mean(all_emergent3.T, axis=0)

std1 = np.std(all_emergent1.T, axis=0)
std2 = np.std(all_emergent2.T, axis=0)
std3 = np.std(all_emergent3.T, axis=0)

In [20]:
## Calculating amplitude responses (ΔF/F)

before = np.mean(matrix1[emergent, :,:], axis = 2)[:, :8]
after = np.mean(matrix1[emergent, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_1 = (after - before)

before = np.mean(matrix2[emergent, :,:], axis = 2)[:, :8]
after = np.mean(matrix2[emergent, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_2 = (after - before)

before = np.mean(matrix3[emergent, :,:], axis = 2)[:, :8]
after = np.mean(matrix3[emergent, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_3 = (after - before)

In [21]:
df1 = pd.DataFrame(amp_1)
df2 = pd.DataFrame(amp_2)
df3 = pd.DataFrame(amp_3)

df1['Type'] = ['P1']*amp_1.shape[0]
df1 = df1.rename(columns={0: 'Amplitude'})
df2['Type'] = ['P2']*amp_2.shape[0]
df2 = df2.rename(columns={0: 'Amplitude'})
df3['Type'] = ['P1+P2']*amp_3.shape[0]
df3 = df3.rename(columns={0: 'Amplitude'})

dfs = pd.concat([df1, df2, df3], axis=0)

In [ ]:
## Plotting mean amplitudes in swarmplots
fig, ax = plt.subplots(figsize = (2.5,4.2))
plt.rcParams["figure.figsize"] = (2.5, 4.2)
my_pal = {"P1+P2": "mediumseagreen", "P1": "gray", "P2": "royalblue"}

ax = sns.swarmplot(x="Type", y="Amplitude", data=dfs, size = 5, palette = my_pal, alpha = 1.0, edgecolors = 'black')
 
plt.xticks(rotation=30, fontsize = 15)
plt.yticks(fontsize = 15)
ax.spines[['right', 'top']].set_visible(False)

ax.set_ylabel('Av. Amp. (ΔF/F)', fontsize = 17)
ax.set(xlabel=None)

df_mean = dfs.groupby('Type', sort=False)['Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--') 

y_max = round(ax.get_ylim()[1])
ax.text(0.25, y_max - 1, 'NS', fontsize = 15)
ax.text(1.35, y_max - 1, '****', fontsize = 15)
ax.text(-0.2, y_max + 1, '-------- **** --------', fontsize = 15)

fig.savefig(FIGURE_4_RESULTS_DIR + '/emergent_amplitudes.png', bbox_inches="tight", dpi = 200)


In [ ]:
## Plotting mean amplitudes in violinplots
fig, ax = plt.subplots(figsize = (2.5,4.2))
plt.rcParams["figure.figsize"] = (2.5, 4.2)

ax = sns.violinplot(x="Type", y="Amplitude", data=dfs, palette = my_pal, alpha = 1.0)
 
plt.xticks(rotation=30, fontsize = 15)
plt.yticks(fontsize = 15)
ax.spines[['right', 'top']].set_visible(False)

ax.set_ylabel('Av. Amp. (ΔF/F)', fontsize = 17)
ax.set(xlabel=None)

df_mean = dfs.groupby('Type', sort=False)['Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--') 

y_max = round(ax.get_ylim()[1])
ax.text(0.25, y_max - 1, 'NS', fontsize = 15)
ax.text(1.35, y_max - 1, '****', fontsize = 15)
ax.text(-0.2, y_max + 1, '-------- **** --------', fontsize = 15)

fig.savefig(FIGURE_4_RESULTS_DIR + '/emergent_amplitudes_violin.png', bbox_inches="tight", dpi = 200)


In [24]:
## Hypothesis tests
## Notice that P-values results were calculated and then they were manually written in every plot
## Therefore they should be edited for each combination of parameters computed
test12 = stats.ttest_rel(df1["Amplitude"], df2["Amplitude"])
test13 = stats.ttest_rel(df1["Amplitude"], df3["Amplitude"])  
test23 = stats.ttest_rel(df2["Amplitude"], df3["Amplitude"]) 

In [ ]:
print(test12)
print(test13)
print(test23)

In [ ]:

f_oneway(df1["Amplitude"], df2["Amplitude"])

In [ ]:
std3

In [ ]:
## Plotting response over time for P1, P2 and combination of both

plt.rcParams['figure.figsize'] = (4,4.2)
fig, ax = plt.subplots()


p1 = gaussian_filter(mean1, sigma=1.0)
p2 = gaussian_filter(mean2, sigma=1.0)
p3 = gaussian_filter(mean3, sigma=1.0)

std1 = gaussian_filter(std1, sigma=1.0)
std2 = gaussian_filter(std2, sigma=1.0)
std3 = gaussian_filter(std3, sigma=1.0)

x = matrix1.shape[1]
x = np.arange(0, x, 1)

ax.errorbar(x, p1, fmt='-o', c = 'gray', label = 'P1', mfc='white', markeredgewidth = 2)
ax.errorbar(x, p2, fmt='-o', c = 'royalblue', label = 'P2', mfc='white', markeredgewidth = 2)
ax.errorbar(x, p3, fmt='-o', c = 'mediumseagreen', label = 'P1+P2', mfc='white', markeredgewidth = 2)
ax.fill_between(x, p1-std1, p1+std1, alpha=0.5, edgecolor='gray', facecolor='gray')
ax.fill_between(x, p2-std2, p2+std2, alpha=0.5, edgecolor='royalblue', facecolor='royalblue')
ax.fill_between(x, p3-std3, p3+std3, alpha=0.5, edgecolor='mediumseagreen', facecolor='mediumseagreen')

ax.plot(x, p1, c = 'gray')
ax.plot(x, p2, c = 'royalblue')
ax.plot(x, p3, c ='mediumseagreen')
 
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
ax.set_xlabel("Time (s)", fontsize = 17)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10))
ax.set_xticklabels([-2, -1, 0, 1, 2])
ax.set_ylabel("Response (ΔF/F)", fontsize = 17)

ax.spines[['right', 'top']].set_visible(False)

fig.suptitle("Emergent DART, avg", fontsize = 17, color = 'red')
fig.savefig(FIGURE_4_RESULTS_DIR + '/all_emergent_time_' + text + '.png', bbox_inches="tight", dpi = 200)

ax.spines[['right', 'top']].set_visible(False)

## EMERGENT GRCS WITH INHIBITION PLOTS

In [29]:
## Calculating emergent cells for the case with inhibition
all_emergent_inh1 = np.mean(matrix1_inh[emergent_inh, :,:], axis = 0)
all_emergent_inh2 = np.mean(matrix2_inh[emergent_inh, :,:], axis = 0)
all_emergent_inh3 = np.mean(matrix3_inh[emergent_inh, :,:], axis = 0)

mean1 = np.mean(all_emergent_inh1.T, axis=0)
mean2 = np.mean(all_emergent_inh2.T, axis=0)
mean3 = np.mean(all_emergent_inh3.T, axis=0)

std1 = np.std(all_emergent_inh1.T, axis=0)
std2 = np.std(all_emergent_inh2.T, axis=0)
std3 = np.std(all_emergent_inh3.T, axis=0)

In [30]:
before = np.mean(matrix1_inh[emergent_inh, :,:], axis = 2)[:, :8]
after = np.mean(matrix1_inh[emergent_inh, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_1 = (after - before)

before = np.mean(matrix2_inh[emergent_inh, :,:], axis = 2)[:, :8]
after = np.mean(matrix2_inh[emergent_inh, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_2 = (after - before)

before = np.mean(matrix3_inh[emergent_inh, :,:], axis = 2)[:, :8]
after = np.mean(matrix3_inh[emergent_inh, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_3 = (after - before)

In [31]:
df1 = pd.DataFrame(amp_1)
df2 = pd.DataFrame(amp_2)
df3 = pd.DataFrame(amp_3)

df1['Type'] = ['P1']*amp_1.shape[0]
df1 = df1.rename(columns={0: 'Amplitude'})
df2['Type'] = ['P2']*amp_2.shape[0]
df2 = df2.rename(columns={0: 'Amplitude'})
df3['Type'] = ['P1+P2']*amp_3.shape[0]
df3 = df3.rename(columns={0: 'Amplitude'})

dfs = pd.concat([df1, df2, df3], axis=0)

In [ ]:
fig, ax = plt.subplots(figsize = (2.5,4.2))
plt.rcParams["figure.figsize"] = (3,4.2)

ax = sns.swarmplot(x="Type", y="Amplitude", data=dfs, size = 5, palette = my_pal, alpha = 1.0, edgecolors = 'black')
 
plt.xticks(rotation=30, fontsize = 15)
plt.yticks(fontsize = 15)
ax.spines[['right', 'top']].set_visible(False)

ax.set_ylabel('Avg. Amp. (ΔF/F)', fontsize = 17)
ax.set(xlabel=None)

df_mean = dfs.groupby('Type', sort=False)['Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--') 
y_max = round(ax.get_ylim()[1])
ax.text(0.25, y_max - 1, '*', fontsize = 15)
ax.text(1.35, y_max - 1, '****', fontsize = 15)
ax.text(-0.2, y_max + 1, '--------- **** ---------', fontsize = 15)

fig.savefig(FIGURE_4_RESULTS_DIR + '/emergent_inh_amplitudes.png', bbox_inches="tight", dpi = 200)


In [ ]:
fig, ax = plt.subplots(figsize = (2.5,4.2))
plt.rcParams["figure.figsize"] = (3,4.2)

ax = sns.violinplot(x="Type", y="Amplitude", data=dfs, palette = my_pal, alpha = 1.0)
 
plt.xticks(rotation=30, fontsize = 15)
plt.yticks(fontsize = 15)
ax.spines[['right', 'top']].set_visible(False)

ax.set_ylabel('Avg. Amp. (ΔF/F)', fontsize = 17)
ax.set(xlabel=None)

df_mean = dfs.groupby('Type', sort=False)['Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--') 

y_max = round(ax.get_ylim()[1])
ax.text(0.25, y_max - 1, '*', fontsize = 15)
ax.text(1.35, y_max - 1, '****', fontsize = 15)
ax.text(-0.2, y_max + 1, '--------- **** ---------', fontsize = 15)


fig.savefig(FIGURE_4_RESULTS_DIR + '/emergent_inh_amplitudes_violin.png', bbox_inches="tight", dpi = 200)


In [ ]:

test12 = stats.ttest_rel(df1["Amplitude"], df2["Amplitude"])
test13 = stats.ttest_rel(df1["Amplitude"], df3["Amplitude"])  
test23 = stats.ttest_rel(df2["Amplitude"], df3["Amplitude"]) 
print(test12)
print(test13)
print(test23)

In [ ]:

f_oneway(df1["Amplitude"], df2["Amplitude"])

In [ ]:
plt.rcParams['figure.figsize'] = (4,4.2)
fig, ax = plt.subplots()

p1 = gaussian_filter(mean1, sigma=1.0)
p2 = gaussian_filter(mean2, sigma=1.0)
p3 = gaussian_filter(mean3, sigma=1.0)

std1 = gaussian_filter(std1, sigma=1.0)
std2 = gaussian_filter(std2, sigma=1.0)
std3 = gaussian_filter(std3, sigma=1.0)

x = matrix1.shape[1]
x = np.arange(0, x, 1)

ax.errorbar(x, p1, fmt='-o', c = 'gray', label = 'P1', mfc='white', markeredgewidth = 2)
ax.errorbar(x, p2, fmt='-o', c = 'royalblue', label = 'P2', mfc='white', markeredgewidth = 2)
ax.errorbar(x, p3, fmt='-o', c = 'mediumseagreen', label = 'P1+P2', mfc='white', markeredgewidth = 2)
ax.fill_between(x, p1-std1, p1+std1, alpha=0.5, edgecolor='gray', facecolor='gray')
ax.fill_between(x, p2-std2, p2+std2, alpha=0.5, edgecolor='royalblue', facecolor='royalblue')
ax.fill_between(x, p3-std3, p3+std3, alpha=0.5, edgecolor='mediumseagreen', facecolor='mediumseagreen')

ax.plot(x, p1, c = 'gray')
ax.plot(x, p2, c = 'royalblue')
ax.plot(x, p3, c ='mediumseagreen')
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
ax.set_xlabel("Time (s)", fontsize = 17)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10))
ax.set_xticklabels([-2, -1, 0, 1, 2])
ax.set_ylabel("Response (ΔF/F)", fontsize = 17)
ax.legend(fontsize = 12, framealpha=0.1)

ax.spines[['right', 'top']].set_visible(False)

fig.suptitle("Emergent control, avg", fontsize = 17)
fig.savefig(FIGURE_4_RESULTS_DIR + '/all_emergent_time_inh_' + text + '.png', bbox_inches="tight", dpi = 200)

ax.spines[['right', 'top']].set_visible(False)

## SUPRESSED GRCS WITH INHIBITION PLOTS

In [37]:
## Calculating suppressed cells for the case with inhibition

In [38]:
supressed_inh1 = np.mean(matrix1_inh[supressed_inh, :,:], axis = 0)
supressed_inh2 = np.mean(matrix2_inh[supressed_inh, :,:], axis = 0)
supressed_inh3 = np.mean(matrix3_inh[supressed_inh, :,:], axis = 0)

mean1 = np.mean(supressed_inh1.T, axis=0)
mean2 = np.mean(supressed_inh2.T, axis=0)
mean3 = np.mean(supressed_inh3.T, axis=0)

std1 = np.std(supressed_inh1.T, axis=0)
std2 = np.std(supressed_inh2.T, axis=0)
std3 = np.std(supressed_inh3.T, axis=0)

In [39]:
before = np.mean(matrix1_inh[supressed_inh, :,:], axis = 2)[:, :8]
after = np.mean(matrix1_inh[supressed_inh, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_1 = (after - before)

before = np.mean(matrix2_inh[supressed_inh, :,:], axis = 2)[:, :8]
after = np.mean(matrix2_inh[supressed_inh, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_2 = (after - before)

before = np.mean(matrix3_inh[supressed_inh, :,:], axis = 2)[:, :8]
after = np.mean(matrix3_inh[supressed_inh, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_3 = (after - before)

In [40]:
df1 = pd.DataFrame(amp_1)
df2 = pd.DataFrame(amp_2)
df3 = pd.DataFrame(amp_3)

df1['Type'] = ['P1']*amp_1.shape[0]
df1 = df1.rename(columns={0: 'Amplitude'})
df2['Type'] = ['P2']*amp_2.shape[0]
df2 = df2.rename(columns={0: 'Amplitude'})
df3['Type'] = ['P1+P2']*amp_3.shape[0]
df3 = df3.rename(columns={0: 'Amplitude'})

dfs = pd.concat([df1, df2, df3], axis=0)

In [ ]:
fig, ax = plt.subplots(figsize = (2.5,4.2))
plt.rcParams["figure.figsize"] = (3,4.2)

ax = sns.swarmplot(x="Type", y="Amplitude", data=dfs, size = 5, palette = my_pal, alpha = 1.0, edgecolors = 'black')
 
plt.xticks(rotation=30, fontsize = 15)
plt.yticks(fontsize = 15)
ax.spines[['right', 'top']].set_visible(False)

ax.set_ylabel('Avg. Amp. (ΔF/F)', fontsize = 17)
ax.set(xlabel=None)
df_mean = dfs.groupby('Type', sort=False)['Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--') 

y_max = round(ax.get_ylim()[1])
ax.text(0.25, y_max - 1, '****', fontsize = 15)
ax.text(1.35, y_max - 1, '****', fontsize = 15)
ax.text(-0.2, y_max + 1, '--------- NS ---------', fontsize = 15)

fig.savefig(FIGURE_4_RESULTS_DIR + '/suppressed_inh_amplitudes.png', bbox_inches="tight", dpi = 200)


In [ ]:
fig, ax = plt.subplots(figsize = (2.5,4.2))
plt.rcParams["figure.figsize"] = (3,4.2)

ax = sns.violinplot(x="Type", y="Amplitude", data=dfs, palette = my_pal, alpha = 1.0)
 
plt.xticks(rotation=30, fontsize = 15)
plt.yticks(fontsize = 15)
ax.spines[['right', 'top']].set_visible(False)

ax.set_ylabel('Avg. Amp. (ΔF/F)', fontsize = 17)
ax.set(xlabel=None)
df_mean = dfs.groupby('Type', sort=False)['Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--') 

y_max = round(ax.get_ylim()[1])
ax.text(0.25, y_max - 2, '****', fontsize = 15)
ax.text(1.35, y_max - 2, '****', fontsize = 15)
ax.text(-0.2, y_max + 1, '--------- NS ---------', fontsize = 15)

fig.savefig(FIGURE_4_RESULTS_DIR + '/suppressed_inh_amplitudes_violin.png', bbox_inches="tight", dpi = 200)


In [ ]:

test12 = stats.ttest_rel(df1["Amplitude"], df2["Amplitude"])
test13 = stats.ttest_rel(df1["Amplitude"], df3["Amplitude"])  
test23 = stats.ttest_rel(df2["Amplitude"], df3["Amplitude"]) 
print(test12)
print(test13)
print(test23)

In [ ]:

f_oneway(df1["Amplitude"], df3["Amplitude"])

In [ ]:
plt.rcParams['figure.figsize'] = (4,4.2)
fig, ax = plt.subplots()


p1 = gaussian_filter(mean1, sigma=1.0)
p2 = gaussian_filter(mean2, sigma=1.0)
p3 = gaussian_filter(mean3, sigma=1.0)

std1 = gaussian_filter(std1, sigma=1.0)
std2 = gaussian_filter(std2, sigma=1.0)
std3 = gaussian_filter(std3, sigma=1.0)

x = matrix1.shape[1]
x = np.arange(0, x, 1)

ax.errorbar(x, p1, fmt='-o', c = 'gray', label = 'P1', mfc='white', markeredgewidth = 2)
ax.errorbar(x, p2, fmt='-o', c = 'royalblue', label = 'P2',mfc='white', markeredgewidth = 2)
ax.errorbar(x, p3, fmt='-o', c = 'mediumseagreen', label = 'P1+P2', mfc='white', markeredgewidth = 2)
ax.fill_between(x, p1-std1, p1+std1, alpha=0.5, edgecolor='gray', facecolor='gray')
ax.fill_between(x, p2-std2, p2+std2, alpha=0.5, edgecolor='royalblue', facecolor='royalblue')
ax.fill_between(x, p3-std3, p3+std3, alpha=0.5, edgecolor='mediumseagreen', facecolor='mediumseagreen')

ax.plot(x, p1, c = 'gray')
ax.plot(x, p2, c = 'royalblue')
ax.plot(x, p3, c ='mediumseagreen')
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
ax.set_xlabel("Time (s)", fontsize = 17)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10))
ax.set_xticklabels([-2, -1, 0, 1, 2])
ax.set_ylabel("Response (ΔF/F)", fontsize = 17)

ax.spines[['right', 'top']].set_visible(False)

fig.suptitle("Suppressed control, avg", fontsize = 17)
fig.savefig(FIGURE_4_RESULTS_DIR + '/all_supressed_time_inh_' + text + '.png', bbox_inches="tight", dpi = 200)

ax.spines[['right', 'top']].set_visible(False)

## SUPRESSED GRCS PLOTS

In [46]:
## Calculating suppressed cells for the case without inhibition

supressed_1 = np.mean(matrix1[supressed, :,:], axis = 0)
supressed_2 = np.mean(matrix2[supressed, :,:], axis = 0)
supressed_3 = np.mean(matrix3[supressed, :,:], axis = 0)

mean1 = np.mean(supressed_1.T, axis=0)
mean2 = np.mean(supressed_2.T, axis=0)
mean3 = np.mean(supressed_3.T, axis=0)

std1 = np.std(supressed_1.T, axis=0)
std2 = np.std(supressed_2.T, axis=0)
std3 = np.std(supressed_3.T, axis=0)

In [47]:
before = np.mean(matrix1[supressed, :,:], axis = 2)[:, :8]
after = np.mean(matrix1[supressed, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_1 = (after - before)

before = np.mean(matrix2[supressed, :,:], axis = 2)[:, :8]
after = np.mean(matrix2[supressed, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_2 = (after - before)

before = np.mean(matrix3[supressed, :,:], axis = 2)[:, :8]
after = np.mean(matrix3[supressed, :,:], axis = 2)[:, 8:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
amp_3 = (after - before)

In [48]:
df1 = pd.DataFrame(amp_1)
df2 = pd.DataFrame(amp_2)
df3 = pd.DataFrame(amp_3)

df1['Type'] = ['P1']*amp_1.shape[0]
df1 = df1.rename(columns={0: 'Amplitude'})
df2['Type'] = ['P2']*amp_2.shape[0]
df2 = df2.rename(columns={0: 'Amplitude'})
df3['Type'] = ['P1+P2']*amp_3.shape[0]
df3 = df3.rename(columns={0: 'Amplitude'})

dfs = pd.concat([df1, df2, df3], axis=0)

In [ ]:
fig, ax = plt.subplots(figsize = (2.5,4.2))
plt.rcParams["figure.figsize"] = (3,4.2)

ax = sns.swarmplot(x="Type", y="Amplitude", data=dfs, size = 5, palette = my_pal, alpha = 1.0, edgecolors = 'black')
 
plt.xticks(rotation=30, fontsize = 15)
plt.yticks(fontsize = 15)
ax.spines[['right', 'top']].set_visible(False)

ax.set_ylabel('Avg. Amp. (ΔF/F)', fontsize = 17)

df_mean = dfs.groupby('Type', sort=False)['Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--') 
ax.set(xlabel=None)
y_max = round(ax.get_ylim()[1])
ax.text(0.25, y_max - 0.5, '****', fontsize = 15)
ax.text(1.35, y_max - 0.5, '****', fontsize = 15)
ax.text(-0.2, y_max + 0.5, '--------- **** ---------', fontsize = 15)

fig.savefig(FIGURE_4_RESULTS_DIR + '/suppressed_amplitudes.png', bbox_inches="tight", dpi = 200)


In [ ]:

test12 = stats.ttest_rel(df1["Amplitude"], df2["Amplitude"])
test13 = stats.ttest_rel(df1["Amplitude"], df3["Amplitude"])  
test23 = stats.ttest_rel(df2["Amplitude"], df3["Amplitude"]) 
print(test12)
print(test13)
print(test23)

In [ ]:

f_oneway(df2["Amplitude"], df3["Amplitude"])

In [ ]:
fig, ax = plt.subplots(figsize = (2.5,4.2))
plt.rcParams["figure.figsize"] = (3,4.2)

ax = sns.violinplot(x="Type", y="Amplitude", data=dfs, palette = my_pal, alpha = 1.0)
 
plt.xticks(rotation=30, fontsize = 15)
plt.yticks(fontsize = 15)
ax.spines[['right', 'top']].set_visible(False)

ax.set_ylabel('Avg. Amp. (ΔF/F)', fontsize = 17)

df_mean = dfs.groupby('Type', sort=False)['Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--') 
ax.set(xlabel=None)
y_max = round(ax.get_ylim()[1])
ax.text(0.25, y_max - 0.5, '****', fontsize = 15)
ax.text(1.35, y_max - 0.5, '****', fontsize = 15)
ax.text(-0.2, y_max + 0.5, '--------- **** ---------', fontsize = 15)

fig.savefig(FIGURE_4_RESULTS_DIR + '/suppressed_amplitudes_violin.png', bbox_inches="tight", dpi = 200)


In [ ]:
plt.rcParams['figure.figsize'] = (4,4.2)
fig, ax = plt.subplots()


p1 = gaussian_filter(mean1, sigma=1.0)
p2 = gaussian_filter(mean2, sigma=1.0)
p3 = gaussian_filter(mean3, sigma=1.0)

std1 = gaussian_filter(std1, sigma=1.0)
std2 = gaussian_filter(std2, sigma=1.0)
std3 = gaussian_filter(std3, sigma=1.0)

x = matrix1.shape[1]
x = np.arange(0, x, 1)

ax.errorbar(x, p1, fmt='-o', c = 'gray', label = 'P1',mfc='white', markeredgewidth = 2)
ax.errorbar(x, p2, fmt='-o', c = 'royalblue', label = 'P2', mfc='white', markeredgewidth = 2)
ax.errorbar(x, p3, fmt='-o', c = 'mediumseagreen', label = 'P1+P2', mfc='white', markeredgewidth = 2)
ax.fill_between(x, p1-std1, p1+std1, alpha=0.5, edgecolor='gray', facecolor='gray')
ax.fill_between(x, p2-std2, p2+std2, alpha=0.5, edgecolor='royalblue', facecolor='royalblue')
ax.fill_between(x, p3-std3, p3+std3, alpha=0.5, edgecolor='mediumseagreen', facecolor='mediumseagreen')

ax.plot(x, p1, c = 'gray')
ax.plot(x, p2, c = 'royalblue')
ax.plot(x, p3, c ='mediumseagreen')
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
ax.set_xlabel("Time (s)", fontsize = 17)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10))
ax.set_xticklabels([-2, -1, 0, 1, 2])
ax.set_ylabel("Response (ΔF/F)", fontsize = 17)

ax.spines[['right', 'top']].set_visible(False)

fig.suptitle("Suppressed DART, avg", fontsize = 17, color = 'red')

fig.savefig(FIGURE_4_RESULTS_DIR + '/all_supressed_time_' + text + '.png', bbox_inches="tight", dpi = 200)

ax.spines[['right', 'top']].set_visible(False) 